In [1]:
import warnings

warnings.filterwarnings('ignore')

import os
import random
import optuna
import numpy as np
import pandas as pd
from tab_transformer_pytorch import TabTransformer
import scipy.stats as st
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, StandardScaler, OneHotEncoder
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = False  # type: ignore

seed_everything()   

In [3]:
df = pd.read_csv('./impute_set/imp3.csv')

class CustomDataset(Dataset):
    def __init__(self, x, y, cat_col, numeric_col):
        self.X_cat = np.array(x[cat_col]).astype(np.int32)
        self.X_num = np.array(x[numeric_col]).astype(np.float32)
        self.y = np.array(y).astype(np.float32)
        
    def __len__(self):
        return len(self.X_cat)
    
    def __getitem__(self, idx):
        X_cat = torch.from_numpy(self.X_cat[idx])
        X_num = torch.from_numpy(self.X_num[idx])
        y = torch.from_numpy(self.y[idx])
        return X_cat, X_num, y


def preprocessing(df, numeric='minmax', category='label'):
    X = df.drop('BS3_1', axis=1)
    y = df[['BS3_1']]
    numeric_col = [
        'FEV1', 'FEV1FVC', 'age', 'BS6_3', 'BS6_2_1', 'BD1',
        '건강문해력', 'Total_slp_wk', 'EQ_5D', 'BE3_31', 'BE5_1', '질환유병기간'
    ]
    cat_col = []
    for col in X.columns:
        if col not in numeric_col:
            cat_col.append(col)

    df_num, df_cat = X[numeric_col], X[cat_col]
    if numeric == 'minmax':
        n_pre = MinMaxScaler()
    else:
        n_pre = StandardScaler()
    df_num = pd.DataFrame(n_pre.fit_transform(df_num), columns=df_num.columns)

    if category == 'label':
        c_pre = OrdinalEncoder()
        df_cat = pd.DataFrame(c_pre.fit_transform(df_cat), columns=df_cat.columns)
    else:
        c_pre = OneHotEncoder(sparse_output=False)
        df_cat = pd.DataFrame(c_pre.fit_transform(df_cat))

    X = pd.concat([df_num, df_cat], axis=1)
    uniques = []
    for col in cat_col:
        uniques.append(len(X[col].unique()))

    return X, y, uniques, numeric_col, cat_col


# def test_with_imputations(model, train_loader, test_loader, train_X, test_y, numeric_col, uniques):
def test_with_imputations(model, train_loader, test_loader, test_y):
    # combined_array = np.column_stack((train_X[numeric_col].mean().values, train_X[numeric_col].std().values))

    # class_counts = torch.tensor([test_y.value_counts()[0], test_y.value_counts()[1]])
    # class_weights = 1.0 / class_counts
    # class_weights /= class_weights.sum()

    # device = torch.device('cuda')

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_negatives = (test_y == 0).sum()
    num_positives = (test_y == 1).sum()
    class_weights = torch.tensor(num_negatives / num_positives, dtype=torch.float32).to(device)    
    # model = TabTransformer(
    #     categories=tuple(uniques),
    #     num_continuous=len(numeric_col),
    #     dim=64,dim_out=1,depth=6,heads=8,attn_dropout=.1,ff_dropout=.1,mlp_hidden_mults=(4,2),
    #     mlp_act=nn.ReLU(inplace=True),
    #     continuous_mean_std=torch.tensor(combined_array, dtype=torch.float32)
    # )
    model = model.to(device)
    optim = Adam(model.parameters(), lr=.0001)
    # criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])
    criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights)
    best_f1 = 0.0
    best_epoch = 0
    epochs = 50
    for epoch in range(epochs):
        running_loss = 0.0
        model.train()
        for x_cat, x_num, yy in train_loader:
            optim.zero_grad()
            x_cat, x_num, yy = x_cat.to(device), x_num.to(device), yy.to(device)
            preds = model(x_cat, x_num)
            loss = criterion(preds.squeeze(), yy.squeeze())
            loss.backward()
            optim.step()
            running_loss += loss.item()
        # print(f'{epoch+1} Epoch | Loss: {running_loss/len(train_loader):.4f}')

        model.eval()
        val_loss = 0.0
        correct = 0
        val_preds = []
        val_targets = []
        with torch.no_grad():
            for x_cat, x_num, yy in test_loader:
                x_cat, x_num, yy = x_cat.to(device), x_num.to(device), yy.to(device)
                preds = model(x_cat, x_num)
                val_loss += criterion(preds.squeeze(), yy.squeeze()).item()
                yy = yy.detach().cpu().numpy().squeeze()
                preds = torch.sigmoid(preds).detach().cpu().numpy().squeeze()
                pred_labels = np.where(preds>=.5, 1, 0)
                correct += (pred_labels == yy).sum().item()
                
                val_preds.extend(pred_labels.tolist())
                val_targets.extend(yy.tolist())
        val_loss /= len(test_loader)
        val_f1 = f1_score(val_targets, val_preds, average='macro')
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_epoch = epoch+1
            torch.save(model.state_dict(), 'bestTabTransformer.pth')
            
    print(f'Best Epoch: {best_epoch} | Best F1 : {best_f1:.4f}')  
    return best_f1  


def test_with_5fold(df, numeric, category, shuffle=True):
    f1s = []
    X, y, uniques, numeric_col, cat_col = preprocessing(df, numeric, category)
    if shuffle:
        skf = StratifiedKFold(n_splits=5, shuffle=shuffle, random_state=42)
    else:
        skf = StratifiedKFold(n_splits=5, shuffle=shuffle)
    
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        train_X, train_y = X.iloc[train_idx], y.iloc[train_idx]
        test_X, test_y = X.iloc[test_idx], y.iloc[test_idx]

        train_set = CustomDataset(train_X, train_y, cat_col, numeric_col)
        test_set = CustomDataset(test_X, test_y, cat_col, numeric_col)
        train_loader = DataLoader(train_set, batch_size=64, shuffle=True, pin_memory=True)
        test_loader = DataLoader(test_set, batch_size=64, shuffle=True, pin_memory=True)

        f1_value = test_with_imputations(
            train_loader, test_loader, train_X, test_y, numeric_col, uniques
        )
        f1s.append(f1_value)

    return f1s


def get_cv_results(f1s:list):
    f1s = np.array(f1s)
    mean_f1 = np.mean(f1s)
    std_f1 = np.std(f1s)
    ci95 = st.t.interval(.95, df=len(f1s)-1, loc=mean_f1, scale=std_f1/np.sqrt(len(f1s)))
    return mean_f1, std_f1, ci95

In [4]:
def objective(trial:optuna.Trial, df):
    X, y, uniques, numeric_col, cat_col = preprocessing(df, 'standard', 'label')

    scores = []
    param = {
        'dim' : trial.suggest_int('dim', 32, 128),
        'depth' : trial.suggest_int('depth', 2, 8),
        'heads' : trial.suggest_int('heads', 2, 8),
        'dropout' : trial.suggest_float('dropout', 0, 0.5),
        'mlp_hidden_mults' : trial.suggest_categorical('mlp_hidden_mults', [(4,2),(2,1),(4,1),(2,2)]),
        'mlp_act' : trial.suggest_categorical('mlp_act', [nn.ReLU, nn.GELU, nn.LeakyReLU, nn.SiLU]),
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        train_X, train_y = X.iloc[train_idx], y.iloc[train_idx]
        test_X, test_y = X.iloc[test_idx], y.iloc[test_idx]

        train_set = CustomDataset(train_X, train_y, cat_col, numeric_col)
        test_set = CustomDataset(test_X, test_y, cat_col, numeric_col)
        train_loader = DataLoader(train_set, batch_size=64, shuffle=True, pin_memory=True)
        test_loader = DataLoader(test_set, batch_size=64, shuffle=True, pin_memory=True)

        combined_array = np.column_stack((train_X[numeric_col].mean().values, train_X[numeric_col].std().values))

        model = TabTransformer(
            categories=tuple(uniques),
            num_continuous=len(numeric_col),
            dim=param['dim'],dim_out=1,depth=param['depth'],heads=param['heads'],attn_dropout=param['dropout'],ff_dropout=param['dropout'],mlp_hidden_mults=param['mlp_hidden_mults'],
            mlp_act=param['mlp_act'](),
            continuous_mean_std=torch.tensor(combined_array, dtype=torch.float32)
        )

        f1_value = test_with_imputations(
            model, train_loader, test_loader, test_y
        )
        scores.append(f1_value)

    return np.mean(scores)

In [5]:
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial:objective(trial, df), n_trials=100)
print(f'Best trial: {study.best_trial}')
print(f'Best value: {study.best_value}')
print(f'Best params: {study.best_params}')

[I 2025-05-19 14:56:52,617] A new study created in memory with name: no-name-d9680918-3751-492a-8d88-8213f9afb86c


Best Epoch: 7 | Best F1 : 0.7809
Best Epoch: 8 | Best F1 : 0.7843
Best Epoch: 6 | Best F1 : 0.7657
Best Epoch: 1 | Best F1 : 0.7177


[I 2025-05-19 14:57:25,258] Trial 0 finished with value: 0.7719247006319827 and parameters: {'dim': 50, 'depth': 8, 'heads': 3, 'dropout': 0.2248903113692915, 'mlp_hidden_mults': (4, 2), 'mlp_act': <class 'torch.nn.modules.activation.GELU'>}. Best is trial 0 with value: 0.7719247006319827.


Best Epoch: 4 | Best F1 : 0.8111
Best Epoch: 5 | Best F1 : 0.7989
Best Epoch: 29 | Best F1 : 0.7869
Best Epoch: 2 | Best F1 : 0.7304
Best Epoch: 34 | Best F1 : 0.6919


[I 2025-05-19 14:58:05,805] Trial 1 finished with value: 0.7589618848966527 and parameters: {'dim': 96, 'depth': 5, 'heads': 3, 'dropout': 0.49454350556452215, 'mlp_hidden_mults': (2, 2), 'mlp_act': <class 'torch.nn.modules.activation.LeakyReLU'>}. Best is trial 0 with value: 0.7719247006319827.


...

Best Epoch: 16 | Best F1 : 0.7479
Best Epoch: 7 | Best F1 : 0.7843
Best Epoch: 4 | Best F1 : 0.7883
Best Epoch: 8 | Best F1 : 0.7322
Best Epoch: 24 | Best F1 : 0.6651


[I 2025-05-19 15:52:40,300] Trial 98 finished with value: 0.7526691945144317 and parameters: {'dim': 84, 'depth': 6, 'heads': 2, 'dropout': 0.2548321299268963, 'mlp_hidden_mults': (2, 1), 'mlp_act': <class 'torch.nn.modules.activation.LeakyReLU'>}. Best is trial 79 with value: 0.7775805206145763.


Best Epoch: 3 | Best F1 : 0.7935
Best Epoch: 11 | Best F1 : 0.7989
Best Epoch: 39 | Best F1 : 0.7869
Best Epoch: 5 | Best F1 : 0.7659
Best Epoch: 1 | Best F1 : 0.6812


[I 2025-05-19 15:52:59,931] Trial 99 finished with value: 0.7627417794139596 and parameters: {'dim': 39, 'depth': 5, 'heads': 6, 'dropout': 0.2082675143610535, 'mlp_hidden_mults': (4, 2), 'mlp_act': <class 'torch.nn.modules.activation.LeakyReLU'>}. Best is trial 79 with value: 0.7775805206145763.


Best Epoch: 3 | Best F1 : 0.7809
Best trial: FrozenTrial(number=79, state=1, values=[0.7775805206145763], datetime_start=datetime.datetime(2025, 5, 19, 15, 42, 56, 938026), datetime_complete=datetime.datetime(2025, 5, 19, 15, 43, 14, 404827), params={'dim': 36, 'depth': 4, 'heads': 5, 'dropout': 0.39153463752779954, 'mlp_hidden_mults': (4, 2), 'mlp_act': <class 'torch.nn.modules.activation.LeakyReLU'>}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'dim': IntDistribution(high=128, log=False, low=32, step=1), 'depth': IntDistribution(high=8, log=False, low=2, step=1), 'heads': IntDistribution(high=8, log=False, low=2, step=1), 'dropout': FloatDistribution(high=0.5, log=False, low=0.0, step=None), 'mlp_hidden_mults': CategoricalDistribution(choices=((4, 2), (2, 1), (4, 1), (2, 2))), 'mlp_act': CategoricalDistribution(choices=(<class 'torch.nn.modules.activation.ReLU'>, <class 'torch.nn.modules.activation.GELU'>, <class 'torch.nn.modules.activation.LeakyReLU'>, <c

In [ ]:
# Best Epoch: 5 | Best F1 : 0.7638
# Best Epoch: 2 | Best F1 : 0.8599
# Best Epoch: 22 | Best F1 : 0.8039
# Best Epoch: 21 | Best F1 : 0.7479
# Best Epoch: 22 | Best F1 : 0.6651
# [I 2025-05-19 15:43:14,405] Trial 79 finished with value: 0.7775805206145763 and parameters: 
# # {'dim': 36, 
# 'depth': 4, 
# 'heads': 5, 
# 'dropout': 0.39153463752779954, 
# 'mlp_hidden_mults': (4, 2), 
# 'mlp_act': <class 'torch.nn.modules.activation.LeakyReLU'>}. Best is trial 79 with value: 0.7775805206145763.

In [1]:
def get_cv_results(f1s:list):
    import numpy as np
    import scipy.stats as st
    f1s = np.array(f1s)
    mean_f1 = np.mean(f1s)
    std_f1 = np.std(f1s)
    ci95 = st.t.interval(.95, df=len(f1s)-1, loc=mean_f1, scale=std_f1/np.sqrt(len(f1s)))
    return mean_f1, std_f1, ci95

In [2]:
mean_f1, std_f1, ci95 = get_cv_results([0.7638, 0.8599, 0.8039, 0.7479, 0.6651])
print(f'CV Results: Mean {mean_f1:.2f} | Std {std_f1:.2f} | CI95% {ci95[0]:.2f}~{ci95[1]:.2f}')

CV Results: Mean 0.77 | Std 0.06 | CI95% 0.69~0.85
